# 🧼 Data Preprocessing Pipeline

This notebook provides a comprehensive data preprocessing pipeline for the transport dataset from the year $2022$. The main goal is to clean and prepare the data for further analysis, ensuring its consistency and quality.

The preprocessing pipeline includes the following steps:

1. Loading data from the given files
2. Translating column names
3. Processing product IDs
4. Processing transport types
5. Downloading and loading BAV list (Betriebszentrale AV)
6. Processing stop data
7. Extracting stop data
8. Processing arrival and departure times
9. Handling inconsistent rows
10. Extracting operator data
11. Deleting unnecessary columns
12. Adding delay columns
13. Saving preprocessed data

For memory considerations, the data is processed and saved day by day. A second step is performed using Spark, which is not included in this notebook, but can be found in the [📁 spark](../../spark) folder.

Furthermore, data about operators and stops is provided in separate files. These files are also loaded and processed in this notebook.

The processed data is saved in the `data/preprocessed` folder, as three separate files:
- `data/preprocessed/operators.csv`
- `data/preprocessed/stops.csv`
- `data/preprocessed/transports.parquet`

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import sys
import os

path_to_preprocessing = os.path.join('..', '..', 'src')
sys.path.insert(0, path_to_preprocessing)

from preprocessing.preprocessing_pipeline import preprocess_files
from preprocessing.merge_data import merge_and_save_operator_data, merge_and_save_stop_data, merge_and_save_transport_data

In [95]:
preprocess_files(
    overwrite_existing_file=True,
    print_progress=False
    )

📁 Found 253 valid file(s) in the data folder.


In [130]:
merge_and_save_operator_data()

In [167]:
merge_and_save_stop_data()